In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Make the notebook full(ish) width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Use NumPy and Pandas cos they're brilliant
import numpy as np
import pandas as pd

# Load the code into "tc"
import oshe as tc

Failed to load paths from C:\Users\tgerrish\Miniconda3\lib\site-packages\honeybee\config.json.
Path to radiance is set to: c:/radiance


C:\Users\tgerrish\Miniconda3\lib\site-packages\windrose\windrose.py:29: MatplotlibDeprecationWarning: 
The Appender class was deprecated in Matplotlib 3.1 and will be removed in 3.3.
  addendum = docstring.Appender(msg, "\n\n")
C:\Users\tgerrish\Miniconda3\lib\site-packages\windrose\windrose.py:30: MatplotlibDeprecationWarning: 
The copy_dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use docstring.copy() and cbook.dedent() instead.
  return lambda func: addendum(docstring.copy_dedent(base)(func))
C:\Users\tgerrish\Miniconda3\lib\site-packages\windrose\windrose.py:30: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.getdoc() instead.
  return lambda func: addendum(docstring.copy_dedent(base)(func))
C:\Users\tgerrish\Miniconda3\lib\site-packages\windrose\windrose.py:30: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use 

In [6]:
# Set the EPW path to be processed
epw_path = r"C:\Users\tgerrish\Downloads\SAU_TB_Wajh.AP.404000_TMYx.2004-2018\SAU_TB_Wajh.AP.404000_TMYx.2004-2018.epw"

# Load the weather data into a Pandas DataFrame
df = tc.helpers.load_weather(epw_path)

# Set the output directory where the plots will be saved
output_directory = "../plots"
out_dir = tc.helpers.create_directory(output_directory)

# Create adjusted DBT using 70% effective evaporative cooling in ideal situation
evaporatively_cooled_dbt = df.dry_bulb_temperature - (0.7 * (df.dry_bulb_temperature - df.wet_bulb_temperature))

In [7]:
##########################################################################
# RUN THIS CELL TO GENERATE THE UTCI's FOR DIFFERENT TYPES OF CONDITIONS #
##########################################################################

# Run openfield analysis
idd_file = r"C:\openstudio-2.9.0\EnergyPlus\Energy+.idd"

# Get climate variables
dbt = df.dry_bulb_temperature
rh = df.relative_humidity
ws = df.wind_speed
hir = df.horizontal_infrared_radiation_intensity
sun_altitude = df.sun_altitude

# Define ground zone from material (using predefined dictionary of properties)
gnd_mat = tc.material.material_dict["CONCRETE"]
ground_zone = tc.geometry.Ground(gnd_mat, xy=5, depth=1.5, subsurface_size=5)

# Define shade for shaded case
shades = [
    tc.geometry.Shade(vertices=[[5, 5, 3], [5, 5, 0], [5, -5, 0], [5, -5, 3]]),
    tc.geometry.Shade(vertices=[[5, -5, 3], [5, -5, 0], [-5, -5, 0], [-5, -5, 3]]),
    tc.geometry.Shade(vertices=[[-5, -5, 3], [-5, -5, 0], [-5, 5, 0], [-5, 5, 3]]),
    tc.geometry.Shade(vertices=[[-5, 5, 3], [-5, 5, 0], [5, 5, 0], [5, 5, 3]]),
    tc.geometry.Shade(vertices=[[-5, -5, 3], [-5, 5, 3], [5, 5, 3], [5, -5, 3]]),
]

# Run EnergyPlus for unshaded ground surface temperature calculation
of_srf_temp_unshaded = tc.energyplus.run_energyplus(epw_path, idd_file, ground=ground_zone, shades=None, run=True).to_frame().values.T[0]  # Calculate ground surface temperature

# Run Radiance for unshaded incident radiation calculation
of_dir_rad_unshaded, of_dif_rad_unshaded = tc.radiance.run_radiance(epw_path, ground=ground_zone, shades=None, run=True)  # Calculate incident solar direct and diffuse radiation

# Calculate MRT for unshaded case
of_mrt_unshaded = tc.mrt.mean_radiant_temperature(surrounding_surfaces_temperature=of_srf_temp_unshaded, horizontal_infrared_radiation_intensity=hir, diffuse_horizontal_solar=of_dif_rad_unshaded.T[0], direct_normal_solar=of_dir_rad_unshaded.T[0], sun_altitude=sun_altitude, ground_reflectivity=gnd_mat.reflectivity, sky_exposure=1)[0]

# Run EnergyPlus for shaded ground surface temperature calculation
of_srf_temp_shaded = tc.energyplus.run_energyplus(epw_path, idd_file, ground=ground_zone, shades=shades, run=True).to_frame().values.T[0]  # Calculate ground surface temperature

# Run Radiance for shaded incident radiation calculation
of_dir_rad_shaded, of_dif_rad_shaded = tc.radiance.run_radiance(epw_path, ground=ground_zone, shades=shades, run=True)  # Calculate incident solar direct and diffuse radiation

# Calculate MRT for shaded case
of_mrt_shaded = tc.mrt.mean_radiant_temperature(surrounding_surfaces_temperature=of_srf_temp_shaded, horizontal_infrared_radiation_intensity=hir, diffuse_horizontal_solar=of_dif_rad_shaded.T[0], direct_normal_solar=of_dir_rad_shaded.T[0], sun_altitude=sun_altitude, ground_reflectivity=gnd_mat.reflectivity, sky_exposure=0)[0]

# Create UTCI conditions under different measures
utci_comparison = []

for shade in [True, False]:
    for wind in [0, 1, 6]:
        for evap_clg in [True, False]:
            
            if shade:
                mrt = of_mrt_shaded
                shadedness = "Shaded"
            else:
                mrt = of_mrt_unshaded
                shadedness = "Unshaded"
            
            if wind == 0:
                ws = np.array([0])
                windyness = "NoWind"
            elif wind == 1:
                ws = df.wind_speed.values
                windyness = "WeatherfileWind"
            else:
                ws = np.array([6])
                windyness = "6m/sWind"
            
            if evap_clg:
                dbt = evaporatively_cooled_dbt.values
                evapyness = "EvaporativeCooling"
            else:
                dbt = df.dry_bulb_temperature.values
                evapyness = "NoEvaporativeCooling"
            
            title = "{} - {} - {}".format(shadedness, windyness, evapyness)
            
            print(title)
            
            utci = tc.utci.universal_thermal_climate_index(dbt, mrt, ws, df.relative_humidity.values)
            
            utci_comparison.append(
                {
                    "wind": windyness,
                    "shade": shadedness,
                    "evaporative_cooling": evapyness,
                    "utci": utci,
                    "title": title
                }
            )

# Construct a DataFrame containing all the different UTCI conditions
utci_df = pd.DataFrame([i["utci"] for i in utci_comparison], columns=df.index, index=pd.MultiIndex.from_arrays(np.array([[[i["wind"]], [i["shade"]], [i["evaporative_cooling"]]] for i in utci_comparison]).T[0], names=["Wind", "Shade", "EC"])).T

# Print the UTCI top 10 rows
utci_df.head()

Simulation completed
Writing recipe contents to: C:\Users\tgerrish\AppData\Local\Temp\openfield\gridbased_annual
Calculating solar values...
# Number of sun up hours: 4419
Writing sun matrix to C:\Users\tgerrish\AppData\Local\Temp\openfield\gridbased_annual\sky\sunmtx.smx
Direct and diffuse solar radiation simulation completed
Radiance results loaded
Mean radiant temperature calculated [0.16 seconds]
Simulation completed
Writing recipe contents to: C:\Users\tgerrish\AppData\Local\Temp\openfield\gridbased_annual
Calculating solar values...
# Number of sun up hours: 4419
Writing sun matrix to C:\Users\tgerrish\AppData\Local\Temp\openfield\gridbased_annual\sky\sunmtx.smx
Direct and diffuse solar radiation simulation completed
Radiance results loaded
Mean radiant temperature calculated [0.12 seconds]
Shaded - NoWind - EvaporativeCooling
Universal thermal climate index calculated [0.08 seconds]
Shaded - NoWind - NoEvaporativeCooling
Universal thermal climate index calculated [0.09 seconds]


Wind                            NoWind                       \
Shade                           Shaded                        
EC                  EvaporativeCooling NoEvaporativeCooling   
2018-01-01 00:30:00          18.233181            18.989262   
2018-01-01 01:30:00          17.714205            18.662664   
2018-01-01 02:30:00          17.708059            18.755811   
2018-01-01 03:30:00          17.610027            18.560652   
2018-01-01 04:30:00          17.180838            18.303298   

Wind                   WeatherfileWind                       \
Shade                           Shaded                        
EC                  EvaporativeCooling NoEvaporativeCooling   
2018-01-01 00:30:00          16.143650            17.103033   
2018-01-01 01:30:00          15.564653            16.782473   
2018-01-01 02:30:00          15.596057            16.939866   
2018-01-01 03:30:00          15.463038            16.683172   
2018-01-01 04:30:00          14.970917            16.429093   

Wind                          6m/sWind                       \
Shade                           Shaded                        
EC                  EvaporativeCooling NoEvaporativeCooling   
2018-01-01 00:30:00           6.925062             8.475966   
2018-01-01 01:30:00           6.174323             8.165067   
2018-01-01 02:30:00           6.311115             8.506287   
2018-01-01 03:30:00           6.079223             8.072468   
2018-01-01 04:30:00           5.410947             7.819272   

Wind                            NoWind                       \
Shade                         Unshaded                        
EC                  EvaporativeCooling NoEvaporativeCooling   
2018-01-01 00:30:00          16.298572            17.086804   
2018-01-01 01:30:00          15.728380            16.717405   
2018-01-01 02:30:00          15.752998            16.845143   
2018-01-01 03:30:00          15.526841            16.519894   
2018-01-01 04:30:00          15.014369            16.188403   

Wind                   WeatherfileWind                       \
Shade                         Unshaded                        
EC                  EvaporativeCooling NoEvaporativeCooling   
2018-01-01 00:30:00          14.262083            15.255752   
2018-01-01 01:30:00          13.634746            14.895596   
2018-01-01 02:30:00          13.697219            15.088229   
2018-01-01 03:30:00          13.439684            14.704770   
2018-01-01 04:30:00          12.868051            14.380529   

Wind                          6m/sWind                       
Shade                         Unshaded                       
EC                  EvaporativeCooling NoEvaporativeCooling  
2018-01-01 00:30:00           5.172610             6.760411  
2018-01-01 01:30:00           4.381889             6.418322  
2018-01-01 02:30:00           4.550362             6.795952  
2018-01-01 03:30:00           4.202282             6.243120  
2018-01-01 04:30:00           3.465431             5.930408

In [8]:
####################################################
# RUN THIS CELL TO GENERATE ALL THE CLIMATE CHARTS #
####################################################

# Create diurnal profile charts
moistures = ["rh", "wbt", "dpt"]
for m in moistures:
    tc.plot.diurnal(df, moisture=m, save_path=out_dir / "diurnal_{0:}.png".format(m))

# Create heatmaps
_vars = {
    "dry_bulb_temperature": "Reds",
    "global_horizontal_radiation": "Oranges",
    "wind_speed": "Greys",
    "relative_humidity": "Blues",
    "wet_bulb_temperature": "Greens",
    "total_sky_cover": "Purples",
}
for k, v in _vars.items():
    tc.plot.heatmap_generic(df[k], cmap=v, save_path=out_dir / "heatmap_{0:}.png".format(k))

# Create windroses
for s in ["Annual","Spring","Summer","Autumn","Winter","Shoulder"][0:]:
    for d in ["Daily", "Morning", "Midday", "Afternoon", "Evening", "Night", "MorningShoulder", "AfternoonShoulder"][0:]:
        tc.plot.windrose(df, season_period=s, day_period=d, n_sector=16, save_path=out_dir / "windrose_{0:}_{1:}.png".format(d, s), close=True)

# Create wind frequency
for s in ["Annual","Spring","Summer","Autumn","Winter","Shoulder"][0:]:
    for d in ["Daily", "Morning", "Midday", "Afternoon", "Evening", "Night", "MorningShoulder", "AfternoonShoulder"][0:]:
        tc.plot.wind_frequency(df, season_period=s, day_period=d, save_path=out_dir / "windfreq_{0:}_{1:}.png".format(d, s), close=True)

# Plot the UTCI's
for n, i in enumerate(utci_df.columns[0:]):
    tc.charts.utci(utci_df.iloc[:, n].values, save_path=out_dir / "utci_{0:}_{1:}_{2:}.png".format(i[0].replace("/", ""), i[1], i[2]), detailed=True, title="{} - {} - {}".format(i[0], i[1], i[2]), close=True)

Diurnal plot saved to ..\plots\diurnal_rh.png
Diurnal plot saved to ..\plots\diurnal_wbt.png
Diurnal plot saved to ..\plots\diurnal_dpt.png
Heatmap plot saved to ..\plots\heatmap_dry_bulb_temperature.png
Heatmap plot saved to ..\plots\heatmap_global_horizontal_radiation.png
Heatmap plot saved to ..\plots\heatmap_wind_speed.png
Heatmap plot saved to ..\plots\heatmap_relative_humidity.png
Heatmap plot saved to ..\plots\heatmap_wet_bulb_temperature.png
Heatmap plot saved to ..\plots\heatmap_total_sky_cover.png


C:\Users\tgerrish\Documents\GitHub\oshe\oshe\plot.py:1353: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


Windrose saved to ..\plots\windrose_Daily_Annual.png
Windrose saved to ..\plots\windrose_Morning_Annual.png
Windrose saved to ..\plots\windrose_Midday_Annual.png
Windrose saved to ..\plots\windrose_Afternoon_Annual.png
Windrose saved to ..\plots\windrose_Evening_Annual.png
Windrose saved to ..\plots\windrose_Night_Annual.png
Windrose saved to ..\plots\windrose_MorningShoulder_Annual.png
Windrose saved to ..\plots\windrose_AfternoonShoulder_Annual.png
Windrose saved to ..\plots\windrose_Daily_Spring.png
Windrose saved to ..\plots\windrose_Morning_Spring.png
Windrose saved to ..\plots\windrose_Midday_Spring.png
Windrose saved to ..\plots\windrose_Afternoon_Spring.png
Windrose saved to ..\plots\windrose_Evening_Spring.png
Windrose saved to ..\plots\windrose_Night_Spring.png
Windrose saved to ..\plots\windrose_MorningShoulder_Spring.png
Windrose saved to ..\plots\windrose_AfternoonShoulder_Spring.png
Windrose saved to ..\plots\windrose_Daily_Summer.png
Windrose saved to ..\plots\windrose_M

C:\Users\tgerrish\Documents\GitHub\oshe\oshe\charts.py:530: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


Plot saved to ..\plots\utci_NoWind_Shaded_EvaporativeCooling.png
Plot saved to ..\plots\utci_NoWind_Shaded_NoEvaporativeCooling.png
Plot saved to ..\plots\utci_WeatherfileWind_Shaded_EvaporativeCooling.png
Plot saved to ..\plots\utci_WeatherfileWind_Shaded_NoEvaporativeCooling.png
Plot saved to ..\plots\utci_6msWind_Shaded_EvaporativeCooling.png
Plot saved to ..\plots\utci_6msWind_Shaded_NoEvaporativeCooling.png
Plot saved to ..\plots\utci_NoWind_Unshaded_EvaporativeCooling.png
Plot saved to ..\plots\utci_NoWind_Unshaded_NoEvaporativeCooling.png
Plot saved to ..\plots\utci_WeatherfileWind_Unshaded_EvaporativeCooling.png
Plot saved to ..\plots\utci_WeatherfileWind_Unshaded_NoEvaporativeCooling.png
Plot saved to ..\plots\utci_6msWind_Unshaded_EvaporativeCooling.png
Plot saved to ..\plots\utci_6msWind_Unshaded_NoEvaporativeCooling.png


<Figure size 432x432 with 0 Axes>